In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PROJECT_PATH = '/content/drive/My Drive/Projects/Speech-Commands-Classification/'
import sys
sys.path.append(PROJECT_PATH)
sys.path.append('/content/')

In [5]:
!wget -O data.tar.gz https://00f74ba44bade48a7e39a69054c538a35e7d818790-apidata.googleusercontent.com/download/storage/v1/b/download.tensorflow.org/o/data%2Fspeech_commands_v0.02.tar.gz?jk=AFshE3XiGMd7qeVoXIMxQIAOwmJzd4d0wS06vCb8oiB1IICMTdm1F-5QsPbmjIc2UTK_sP2Gl1VB-uMnJNmmGjY5YIuW0J5JJeAzVQFq9kH2XzbzzjdupDvfwsLU6R898YQKkj1uWJSc3E9AOMrffyg7DB0S8w_o6D0ezLkJtauEopDTGa3p2sY6qBDmZG4M9aOFyc31ArSwfecmhOy-WcTXIAgoPtjrNOEcoUhkCpQ5lVrCtX4B73BG3D1D9djwQd5cM3LourSOFBXWUJEv4-DsZeSPSo2JFRl0Op3OOjt5m94lMt1ixg0ZQhMDf2fgbcBMtxYOCIE86Z3lHloYnOcI8jJBBr-CgEL1hdOmeU_90SVRB8DzDzMK7RLaMCszuoVxAhQSv1wp7lK1jVkUXkCXdIsVB1DiYaaoVecK9Zs2j9De6i1Ftpa52MXlOn4QwKuF_ORCnzboDZqAKmiiBf5Ox1xM89ta9Bc8_ON5Z6OJ-WbS6x0M-hMYzz8NgpdO2N7Gy3fJa9rqZtKaSOQqx-7_4L3G_wmK8QV82A97Rr0ktRj4yRgfXqgZwG7rpA4sdxaKq_RfirB3aIPEsZiJbYus3M_jKTm913fhmcBntnlkLPobOGCfly564lJ7kxVyfbZ5bVt9TQ6z8pQZERV5PmuSIsqoKyRlXh6uNyQzrHpsVSnupI6JHAjXd0DuVRrKPhrjYoo8p5qOa04j-4iKGCrqDGnjvPHU6FpdNIzkhZ-8IvMAz9-t6TOKoUtgQ82ztdoLORwbk-ehpxeXi_c96lVNhlvAtNB6l6glXdSi_DQyaZV3J-tykdLEfj5ElvG7B-LUnCgB058EWDWPeTNQ9pdfnq6I-b8CDWWOiqTxoMGWFuSNBKYLZRlRtZAP5yJNXG9iwk6bl0-Uc29dmFPXQyqCSsIQzQY&isca=1

--2020-10-04 09:48:34--  https://00f74ba44bade48a7e39a69054c538a35e7d818790-apidata.googleusercontent.com/download/storage/v1/b/download.tensorflow.org/o/data%2Fspeech_commands_v0.02.tar.gz?jk=AFshE3XiGMd7qeVoXIMxQIAOwmJzd4d0wS06vCb8oiB1IICMTdm1F-5QsPbmjIc2UTK_sP2Gl1VB-uMnJNmmGjY5YIuW0J5JJeAzVQFq9kH2XzbzzjdupDvfwsLU6R898YQKkj1uWJSc3E9AOMrffyg7DB0S8w_o6D0ezLkJtauEopDTGa3p2sY6qBDmZG4M9aOFyc31ArSwfecmhOy-WcTXIAgoPtjrNOEcoUhkCpQ5lVrCtX4B73BG3D1D9djwQd5cM3LourSOFBXWUJEv4-DsZeSPSo2JFRl0Op3OOjt5m94lMt1ixg0ZQhMDf2fgbcBMtxYOCIE86Z3lHloYnOcI8jJBBr-CgEL1hdOmeU_90SVRB8DzDzMK7RLaMCszuoVxAhQSv1wp7lK1jVkUXkCXdIsVB1DiYaaoVecK9Zs2j9De6i1Ftpa52MXlOn4QwKuF_ORCnzboDZqAKmiiBf5Ox1xM89ta9Bc8_ON5Z6OJ-WbS6x0M-hMYzz8NgpdO2N7Gy3fJa9rqZtKaSOQqx-7_4L3G_wmK8QV82A97Rr0ktRj4yRgfXqgZwG7rpA4sdxaKq_RfirB3aIPEsZiJbYus3M_jKTm913fhmcBntnlkLPobOGCfly564lJ7kxVyfbZ5bVt9TQ6z8pQZERV5PmuSIsqoKyRlXh6uNyQzrHpsVSnupI6JHAjXd0DuVRrKPhrjYoo8p5qOa04j-4iKGCrqDGnjvPHU6FpdNIzkhZ-8IvMAz9-t6TOKoUtgQ82ztdoLORwbk-ehpxeXi_c96lVNhlvAtNB6l6glXdS

In [9]:
!tar -xzf /content/data.tar.gz -C /content/data/

In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
import torch
from torch import optim
import torch.nn as nn

import matplotlib.pyplot as plt

from const import Consts
from data_handler.loaders import get_data_loaders
from model import LeNet, weight_init
from train import train, evaluation, test_model

In [14]:
path = '/content/drive/My\ Drive/Projects/Speech-Commands-Classification'
!python {path}/data_handler/create_dataset.py '/content/data' --out_path '/content/dataset/'

In [15]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [18]:
train_loader, valid_loader, test_loader = get_data_loaders('/content/dataset/')

model = LeNet(35)
optimizer = optim.Adam(model.parameters(), lr=Consts.lr)
loss_criterion = nn.NLLLoss()
model.apply(weight_init)
model.to(device)
train_loss = []
val_loss = []
val_acc = []
for epoch in range(Consts.epochs):
    t_loss = train(model, train_loader, optimizer, loss_criterion, device)
    v_loss, v_acc = evaluation(model, valid_loader, loss_criterion, device)
    torch.save(model.state_dict(), f'/content/models/epoch-{epoch + 1}.pth')
    train_loss.append(t_loss)
    val_loss.append(v_loss)__init__() got an unexpected keyword argument 'prefetch_factor'
    val_loss.append(v_loss)__init__() got an unexpected keyword argument 'prefetch_factor'
    val_acc.append(v_acc)
    print(f'train loss in epoch {epoch + 1} is: {t_loss}')
    print(f'validation loss in epoch {epoch + 1} is: {v_loss}')
    print(f'validation accuracy in epoch {epoch + 1} is: {v_acc}')

plot_graphs(train_loss, val_loss, val_acc)

test_model(model, test_loader, loss_criterion, val_loss, device)

  0%|          | 4/2652 [00:41<7:40:59, 10.45s/commands] 


KeyboardInterrupt: ignored